### performance check(Jaccard)

In [1]:
# util
import os
import sys
sys.path.append('../')
import argparse
import asyncio
import json
import random
import pickle
from tqdm import tqdm

# model
import torch
import pandas as pd
import numpy as np


from models.model import ContentBasedRecommender
from models.preprocess import make_new_type_data, make_meta_data, make_type_data

In [2]:
data_dir = '/opt/ml/final-project-level3-recsys-02/data/'
recommender = ContentBasedRecommender(data_dir,'ptnt_embeddings')

In [78]:
place = recommender.map_loader.place
id2place = recommender.id2place
place_id2idx = recommender.place2id
make_new_type_data(place)
# all_df = recommender.ma

,placeID,newType
0,밀밭정원서울마포구마포대로16길13,칼국수
1,식스센스다이닝BAR서울동대문구왕산로2길9(2층방역룸예약),"와인,바"
2,동북양꼬치서울영등포구디지털로37길26-1,양꼬치
3,농부쌈밥서울동작구사당로30길19,쌈밥
4,홍당무김밥서울영등포구문래로180영등포센트럴푸르지오시티,분식
...,...,...
12672,은빛바다광동수산행당동본점서울성동구행당로127-1,"회,생선"
12673,갓파스시분당미금역점경기성남시분당구돌마로67,일식
12674,갓잇송리단길점서울송파구백제고분로45길4-14,서양음식
12675,마니주호프수지점경기용인시수지구수지로342번길17에덴프라자,주류


In [79]:
make_meta_data(place,['placeID', 'menulabel','ageLabel', 'ratingLabel', 'visitLabel', 'blogLabel'])

,placeID,meta
0,밀밭정원서울마포구마포대로16길13,"30000이하['50대', '40대', '60대']4.5이하visitQ2blogQ2"
1,식스센스다이닝BAR서울동대문구왕산로2길9(2층방역룸예약),"30000이상['20대', '30대', '40대']4.5이상visitQ3blogQ4"
2,동북양꼬치서울영등포구디지털로37길26-1,30000이하[]4.5이하visitQ1blogQ1
3,농부쌈밥서울동작구사당로30길19,"10000이하['20대', '30대', '40대']4.5이하visitQ4blogQ4"
4,홍당무김밥서울영등포구문래로180영등포센트럴푸르지오시티,10000이하[]4.5이하visitQ3blogQ1
...,...,...
12670,예송숯불구이서울중구청파로433,10000이하[]4.5이하visitQ3blogQ1
12671,유끼서울강서구강서로5나길109,30000이하[]4.5이하visitQ1blogQ2
12673,갓파스시분당미금역점경기성남시분당구돌마로67,"30000이하['20대', '40대', '30대']4.5이하visitQ4blogQ3"
12674,갓잇송리단길점서울송파구백제고분로45길4-14,"30000이하['20대', '30대', '40대']4.5이상visitQ4blogQ4"


In [80]:
nt_df = make_new_type_data(place)
m_df = make_meta_data(place,['placeID', 'menulabel','ageLabel', 'ratingLabel', 'visitLabel', 'blogLabel'])
p_df = make_type_data(place)
nt_df.columns = ['placeID','feature']
p_df.columns = ['placeID','feature']
m_df.columns = ['placeID','feature']

df = pd.concat([nt_df,p_df,m_df])

,placeID,feature
0,밀밭정원서울마포구마포대로16길13,칼국수
1,식스센스다이닝BAR서울동대문구왕산로2길9(2층방역룸예약),"와인,바"
2,동북양꼬치서울영등포구디지털로37길26-1,양꼬치
3,농부쌈밥서울동작구사당로30길19,쌈밥
4,홍당무김밥서울영등포구문래로180영등포센트럴푸르지오시티,분식
...,...,...
12670,예송숯불구이서울중구청파로433,10000이하[]4.5이하visitQ3blogQ1
12671,유끼서울강서구강서로5나길109,30000이하[]4.5이하visitQ1blogQ2
12673,갓파스시분당미금역점경기성남시분당구돌마로67,"30000이하['20대', '40대', '30대']4.5이하visitQ4blogQ3"
12674,갓잇송리단길점서울송파구백제고분로45길4-14,"30000이하['20대', '30대', '40대']4.5이상visitQ4blogQ4"


In [81]:
def jaccard_similarity(list1, list2): 
    s1 = set(list1)
    s2 = set(list2)
    if len(s1.union(s2)):
        return float(len(s1.intersection(s2)) / len(s1.union(s2)))
    return 0

In [82]:
def get_feat_list(df, place_id, feature):
    return df[df.placeID == place_id][feature].tolist()

In [83]:
def compare_random(place_id, df, column):
    lonlan = place[place.placeID == place_id]['map'].values[0]
    nearest_list = recommender.map_loader.filtermap(lonlan)

    
    m2v_topk = recommender.recommend(lonlan, place_id)
    random_topk = np.array(random.sample(nearest_list, k = min(len(nearest_list), 10)))

    m2v_score, random_score = 0, 0

    target_feat_list = get_feat_list(df, place_id, column)

    for placename in m2v_topk:
        rec_feat_list = get_feat_list(df, placename, column)
        m2v_score += jaccard_similarity(target_feat_list, rec_feat_list)

    for randomname in random_topk:
        rec_feat_list = get_feat_list(df, id2place[randomname], column)
        random_score += jaccard_similarity(target_feat_list, rec_feat_list)

    return m2v_score, random_score


In [84]:
def mean_score():
    mean_m2v, mean_random = 0, 0

    for i in tqdm(id2place.values()):
        m2v_score, random_score = compare_random(i, df, 'feature')
        mean_m2v += m2v_score
        mean_random += random_score

    return mean_m2v, mean_random

In [85]:
mean_score()

100%|██████████| 12677/12677 [10:50<00:00, 19.49it/s]


(16202.652380952699, 5113.423809523769)